# 6장 자연어를 이해하고 생성하는 AI 모델과 API

## 6.1 OpenAI 소개

##  6.2 OpenAI API 사용을 위한 API 키 생성

## 6.3 OpenAI의 모델

## 6.4 OpenAI API 사용하기

### 6.4.1 OpenAI 파이썬 라이브러리 설치

[6장: 156페이지]

In [ ]:
import openai # openai 라이브러리 임포트

openai.api_key = "your_api_key" # OpenAI API의 API 키 지정

In [ ]:
import openai
import os

# API 키 설정
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
import openai
import os

# API 키 설정
openai.api_key = os.environ["OPENAI_API_KEY"]

# 모든 OpenAP API의 모델 리스트를 요청해 가져오기
models = openai.Model.list()

# 가져온 모델 리스트 출력하기 
print(models["data"][0]['id']) # 모델 리스트 중 첫 번째 항목의 ID 출력
# print(models["data"])  # 모델 리스트 전체 정보 출력

### 6.4.2 Chat Completions API를 이용한 챗봇 만들기

[6장: 160페이지]

In [ ]:
import openai

# 대화 메시지 정의
messages = [
    {"role": "user", "content": "대한민국의 수도는 어디인가요?"}
]

# Chat Completions API 호출
response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo", # 모든 사용자 설정 가능
                # model="gpt-3.5-turbo-16k", # gpt-3.5-turbo 대비 4배 긴 토큰 처리
                # model="gpt-4", # GPT-4 모델 사용
                messages=messages, # 전달할 메시지 지정
                max_tokens=1000, # 응답 최대 토큰 수 지정
                temperature=0.8, # 완성의 다양성을 조절하는 온도 설정
                n=1 # 생성할 완성의 개수 지정
)

# 응답 출력
assistant_reply = response.choices[0].message['content'] # 첫 번째 응답 결과 가져오기
print(assistant_reply)

[6장: 161페이지]

In [ ]:
response

[6장: 162페이지]

In [ ]:
import json

message = response.choices[0].message
json.dumps(message, ensure_ascii=False)

In [ ]:
# 대화 메시지 정의
messages = [
    {"role": "user", "content": "한글은 언제 만들어졌나요?"}
]

# Chat Completions API 호출
response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=messages,
                max_tokens=1000,
                temperature=0.8,
                n=2 
)

print("응답 개수:", len(response.choices)) # 응답 개수 출력

[6장: 163페이지]

In [ ]:
print("[응답 0]", response.choices[0].message['content'])
print("[응답 1]", response.choices[1].message['content'])

In [ ]:
import openai

def response_from_ChatAI(user_content, r_num=1):
    
    # 대화 메시지 정의
    messages = [ {"role": "user", "content": user_content} ]

    # Chat Completions API 호출
    response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=messages,
                    max_tokens=1000,
                    temperature=0.8,
                    n=r_num 
    )
    
    # 응답을 리스트에 할당
    assistant_replies = []
    
    for choice in response.choices:
        assistant_replies.append(choice.message['content'])
        
    return assistant_replies # 응답 반환

[6장: 164페이지]

In [ ]:
resp = response_from_ChatAI("대한민국 헌법 제1조 1항은?")
resp

In [ ]:
import textwrap

resps = response_from_ChatAI("ChatGPT는 무엇인가요?", 2) # 두 개의 응답 설정

for resp in resps:
    shorten_resp = textwrap.shorten(resp, 100, placeholder=' [..이하 생략..]')
    print(shorten_resp) # 축약 내용 출력
    print() # 빈 줄 하나를 출력

[6장: 165페이지]

In [ ]:
resps = response_from_ChatAI("두 숫자를 입력받아 더하는 파이썬 함수 만들어 줘")
print(resps[0])

In [ ]:
import openai
import os

openai.api_key = os.environ["OPENAI_API_KEY"] # API 키 설정

user_input = input("AI와 채팅할 내용을 입력하세요 (종료하려면 end를 입력하세요) \n[나]:")

messages = [{"role": "system", "content": "You are a helpful assistant."}]   

ai_message = ""    
while(user_input !="end"):
    message = [{"role": "assistant", "content": ai_message},
               {"role": "user", "content": user_input}]
    messages.extend(message)

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", 
        messages=messages,
        max_tokens=1000,
        temperature=0.9,
        n=1)
    
    ai_message = response.choices[0].message['content']
    print(f"[AI]\n{ai_message}")
    user_input = input("\n[나]:")

if(user_input =="end"):
    print("AI와 채팅을 종료합니다.")

### 6.4.3 Chat Completions API를 이용한 함수 호출

[6장: 168페이지]

In [ ]:
import openai
import json

# 호출 함수
def get_price_info_temp(product_name):
    
    price_info = {
        "product_name": product_name,
        "price": "10,000"
    }
    
    return json.dumps(price_info) # JSON 형식으로 변환해 반환

# Chat Completions API에 사용자 입력과 함수 정보를 보내는 함수
def run_conversation_temp(user_query):   
    # 사용자 입력
    messages = [{"role": "user", "content": user_query}] 
        
    # 함수 정보 입력   
    functions = [                                        
        {
            "name": "get_price_info_temp",
            "description": "제품 이름에 따른 가격 가져오기",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "제품 이름. 예를 들면, 키보드, 마우스",
                    },
                },
                "required": ["product_name"],
            },
        }
    ]
    # 1단계: 사용자 입력과 함수 정보를 Chat Completions API 모델로 보내기    
    response = openai.ChatCompletion.create( # Chat Completions API 모델로 보내기
            model="gpt-3.5-turbo",
            # model="gpt-4",
            messages=messages,
            functions=functions,
            function_call="auto"
    )
    # 2단계: 응답 생성
    response_message = response["choices"][0]["message"] # 응답 메시지
    
    return response_message # 응답 메시지 반환

[6장: 169페이지]

In [ ]:
response_message = run_conversation_temp("대한민국의 수도는 어디인가요?")
json.dumps(response_message, ensure_ascii=False)

[6장: 170페이지]

In [ ]:
user_query = "키보드의 가격은 얼마인가요?"
response_message = run_conversation_temp(user_query)
json.dumps(response_message, ensure_ascii=False)

In [ ]:
user_query = "마우스는 얼마인가요?"
response_message = run_conversation_temp(user_query)
json.dumps(response_message, ensure_ascii=False)

In [ ]:
response_message["function_call"] # response_message.get("function_call") 도 가능

In [ ]:
response_message["function_call"]["name"]

In [ ]:
json.loads(response_message["function_call"]["arguments"])

[6장: 171페이지]

In [ ]:
json.loads(response_message["function_call"]["arguments"])["product_name"]

In [ ]:
import openai
import json

# 호출 함수
def get_price_info(product_name):    
    # 가격 정보
    product_price = {"키보드": "3만원", "마우스": "2만원", "모니터": "30만원"}
    
    prince = product_price.get(product_name) # 제품 이름을 입력해 가격 가져오기
    if prince == None: # 제품 가격이 없을 경우
        prince = "해당 상품은 가격 정보가 없습니다."

    price_info = {
        "product_name": product_name,
        "price": prince
    }
    
    return json.dumps(price_info)

# Chat Completions API를 이용해 사용자 입력에 따라 함수를 호출하고 응답하는 함수
def run_conversation(user_query):
    # 사용자 입력
    messages = [{"role": "user", "content": user_query}] 
        
    # 함수 정보 입력   
    functions = [                                        
        {
            "name": "get_price_info",
            "description": "제품 이름에 따른 가격 가져오기",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "제품 이름. 예를 들면, 키보드, 마우스",
                    },
                },
                "required": ["product_name"],
            },
        }
    ]
    # 1단계: 사용자 입력과 함수 정보를 Chat Completions API 모델로 보내기    
    response = openai.ChatCompletion.create( # Chat Completions API 모델로 보내기
            model="gpt-3.5-turbo",
            # model="gpt-4",
            messages=messages,
            functions=functions,
            function_call="auto"
    )
    # 2단계: 응답 생성
    response_message = response["choices"][0]["message"] # 모델의 응답 메시지
    
    if response_message.get("function_call"): # 응답이 함수 호출인지 확인하기
        # 3단계: JSON 객체를 분석해 함수 이름과 인수를 추출한 후에 함수 호출
        # (주의: JSON 응답이 항상 유효하지 않을 수 있음)
        
        # 함수 이름 추출
        function_name = response_message["function_call"]["name"]
        # 함수 호출을 위한 인수 추출
        function_args = json.loads(response_message["function_call"]["arguments"])
        
        # 함수 호출 및 반환 결과 받기
        function_response = get_price_info(
            product_name=function_args.get("product_name") # 인수 지정
        )
        
        # 4단계: 함수 호출 결과를 기존 메시지에 추가하고,
        #        Chat Completions API 모델로 보내 응답받기

        # 함수 호출 결과를 기존 메시지에 추가하기
        messages.append(response_message)  # 기존 messages에 조력자 응답 추가
        messages.append(                   # 함수와 함수 호출 결과 추가
            {
                "role": "function",           # role: function으로 지정
                "name": function_name,        # name: 호출할 함수 이름 지정
                "content": function_response, # content: 함수 호출 결과 지정
            }
        )
        # 함수 호출 결과를 추가한 메시지를 Chat Completions API 모델로 보내 응답받기
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            # model="gpt-4",
            messages=messages,
        ) 
        return second_response # 두 번째 응답 반환
    
    return response_message # 응답 메시지 반환

[6장: 174페이지]

In [ ]:
user_query = "마우스는 얼마인가요?" # 가격 정보 있음
response = run_conversation(user_query)
json.dumps(response, ensure_ascii=False)

In [ ]:
user_query = "모니터는 얼마인가요?" # 가격 정보 있음
response = run_conversation(user_query)
response["choices"][0]["message"]["content"]

In [ ]:
user_query = "HDD는 얼마인가요?" # 가격 정보 없음
response = run_conversation(user_query)
response["choices"][0]["message"]["content"]

###  6.4.4 Image API를 이용한 이미지 생성

[6장: 177페이지]

In [ ]:
import openai

response = openai.Image.create(
              prompt="Happy robots playing in the playground",
              n=1,
              size="512x512" # 이미지 크기를 512 x 512로 지정  
)

image_url = response['data'][0]['url']
print(image_url) # 이미지 URL 전체 표시

[6장: 178페이지]

In [ ]:
from IPython.display import Image, display

display(Image(image_url, format="png")) # 이미지 링크 표시

In [ ]:
import openai
from IPython.display import Image, display

response = openai.Image.create(
              prompt="Happy robots playing in the playground",
              n=2,
              size="512x512" # 이미지 크기를 512 x 512로 지정 
)

for data in response['data']:
    image_url = data['url'] # 이미지 URL 추출    
    print(image_url[:170]) # 이미지 URL 일부 표시
    # print(image_url) # 이미지 URL 전체 표시
    
    display(Image(image_url, format='png')) # 이미지를 화면에 표시

[6장: 179페이지]

In [ ]:
image_url.split("?")[0].split("/")[-1]

[6장: 180페이지]

In [ ]:
from pathlib import Path

# 디렉터리 경로를 입력해 path 객체를 생성
dir_path = Path('C:/myPyAI/data/download') # 내려받을 폴더 생성   

# 디렉터리가 없다면 생성
dir_path.mkdir(parents=True, exist_ok=True)

# 생성한 디렉터리의 존재 여부 확인
print("{0} 디렉터리의 존재 여부: {1}".format(dir_path, dir_path.exists()))

In [ ]:
import requests

for data in response['data']:
    image_url = data['url'] # 이미지 URL 추출
    
    download_holder = "C:/myPyAI/data/download/" # 다운로드 폴더 지정
    image_filenme = image_url.split("?")[0].split("/")[-1]  # 이미지 파일 이름 추출   
    image_path = download_holder + image_filenme # 다운로드 파일의 경로 생성
    print("이미지 파일 경로:", image_path)
    
    r = requests.get(image_url) # 이미지 URL을 이용해 이미지 가져오기
    with open(image_path, 'wb') as f: # 가져온 이미지를 바이너리 파일로 저장
        f.write(r.content)

[6장: 181페이지]

In [ ]:
import openai
from IPython.display import Image, display

org_image_file = "C:/myPyAI/data/org_image_for_edit.png"   # 원본 이미지
mask_image_file = "C:/myPyAI/data/mask_image_for_edit.png" # 마스크 이미지

# 이미지 편집 생성 
response = openai.Image.create_edit( 
                image=open(org_image_file, "rb"), 
                mask=open(mask_image_file, "rb"),
                prompt="Happy robots swimming in the water",                           
                n=1, # 생성할 이미지 개수 지정
                size="256x256" # 생성할 이미지의 크기 지정                            
)

image_url = response['data'][0]['url'] # 생성 이미지 URL

print("[원본 이미지]")
display(Image(org_image_file, format='png', width=256, height=256)) # 이미지를 화면에 표시
print("[마스크 이미지]")
display(Image(mask_image_file, format='png', width=256, height=256)) # 이미지를 화면에 표시
print("[생성 이미지]")
display(Image(image_url, format='png')) # 이미지를 화면에 표시

[6장: 183페이지]

In [ ]:
import openai
from IPython.display import Image, display

org_image_file = "C:/myPyAI/data/org_image_for_variation.png" # 원본 이미지

# 이미지 변형 생성
response = openai.Image.create_variation( 
                image=open(org_image_file, "rb"),                         
                n=1, # 생성할 이미지 개수 지정
                size="256x256" # 생성할 이미지의 크기 지정                          
)
    
image_url = response['data'][0]['url']

print("[원본 이미지]")
display(Image(org_image_file, format='png', width=256, height=256)) # 이미지를 화면에 표시
print("[생성 이미지]")
display(Image(image_url, format='png')) # 이미지를 화면에 표시

### 6.4.5 Audio API를 이용한 음성 추출

[6장: 186페이지]

In [ ]:
import openai
import os

# 비디오, 오디오 파일 열기
file_path = "C:/myPyAI/data/서연의_하루_TTS_short.mp3"

audio_file = open(file_path, "rb")

# API 키 설정
openai.api_key = os.environ["OPENAI_API_KEY"]

# 지정한 형식으로 음성 추출
response = openai.Audio.transcribe(
            model="whisper-1",
            file=audio_file,
            response_format="text" # text, srt 등 중 하나 선택
            )
audio_file.close()
print(response[:100]) # 일부 출력
# print(response) # 전체 출력

In [ ]:
# 비디오, 오디오 파일 열기
file_path = "C:/myPyAI/data/서연의_하루_TTS_배경음악_short.mp3"
audio_file = open(file_path, "rb")

# 지정한 형식으로 음성 추출
response = openai.Audio.transcribe(
            model="whisper-1",
            file=audio_file,
            response_format="text" # text, srt, vtt, json, verbose_json 중 하나 선택
            )
audio_file.close()
print(response[:100]) # 일부 출력
# print(response) # 전체 출력

[6장: 187페이지]

In [ ]:
file_path = "C:/myPyAI/data/서연의_하루_TTS_배경음악_short.mp3"
audio_file = open(file_path, "rb")

# 지정한 형식으로 음성 추출
response = openai.Audio.translate(
            model="whisper-1",
            file=open(file_path, "rb"),
            response_format="text" # text, srt, vtt, json, verbose_json 중 하나 선택
            )
audio_file.close()
print(response[:100]) # 일부 출력
# print(response) # 전체 출력

### 6.4.6 OpenAI 토큰 계산하기

[6장: 189페이지]

In [ ]:
import tiktoken

text = "tiktoken is great!" # 원본 텍스트
enc = tiktoken.get_encoding("cl100k_base")
# enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

encoded_list = enc.encode(text) # 텍스트 인코딩해 인코딩 리스트 생성
token_num = len(encoded_list)   # 인코딩 리스트의 길이로 토큰 개수 계산
decoded_text = enc.decode(encoded_list) # 인코딩 결과를 디코딩해서 텍스트 복원

print("- 인코딩 결과:", encoded_list)
print("- 토큰 개수:", token_num)
print("- 디코딩 결과:", decoded_text)

## 6.5 정리